In [1]:
from seeq_sysid.app import SYSID
from seeq_sysid.gui.transfer_matrix import TransferMatrix
import pickle
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML, clear_output, display, Javascript

jv = '''$('#header-container').hide();'''
Javascript(jv)

<IPython.core.display.Javascript object>

In [2]:
%%javascript

if (document.addEventListener) {
  document.addEventListener('contextmenu', function(e) {
    e.preventDefault();
  }, false);
} else {
  document.attachEvent('oncontextmenu', function() {
      window.event.returnValue = false;
  });
}

<IPython.core.display.Javascript object>

In [3]:
app = SYSID()

with open('results.pkl', 'rb') as file:
    [arx_train_results, arx_validation_results,
     ss_train_results, ss_validation_results,
     nn_train_results, nn_validation_results] = pickle.load(file)
    

# ARX
app.arx_sheet.mv_select.v_model = ['F_cw']
app.arx_sheet.cv_select.v_model = ['T1', 'T2']
app.arx_sheet.train_condition.v_model = ['Training Condition']

app.arx_sheet.na_min.v_model = 0
app.arx_sheet.nb_min.v_model = 0
app.arx_sheet.nk_max.v_model = 5

app.arx_sheet.panel.identify_model_btn.disabled = False
app.arx_sheet.panel.validate_model_btn.disabled = False
# app.arx_sheet.panel.push_model_btn.disabled = False

app.arx_sheet.canvas.create(train_df=arx_train_results, validation_df=arx_validation_results)

app.arx_sheet.canvas.table.table_header.checkbox_header.v_model = True

app.arx_sheet.canvas.table.items_list[2].style_item.v_model = '_ . _'
app.arx_sheet.canvas.table.items_list[3].style_item.v_model = '_ . _'

app.arx_sheet.canvas.figure.update(2, True, 'dashdot')
app.arx_sheet.canvas.figure.update(3, True, 'dashdot')



# Subspace
app.ss_sheet.mv_select.v_model = ['F_cw']
app.ss_sheet.cv_select.v_model = ['T1', 'T2']
app.ss_sheet.train_condition.v_model = ['Training Condition']

app.ss_sheet.panel.identify_model_btn.disabled = False
app.ss_sheet.panel.validate_model_btn.disabled = False

app.ss_sheet.canvas.create(train_df=ss_train_results, validation_df=ss_validation_results)

app.ss_sheet.canvas.table.table_header.checkbox_header.v_model = True

app.ss_sheet.canvas.table.items_list[2].style_item.v_model = '_ . _'
app.ss_sheet.canvas.table.items_list[3].style_item.v_model = '_ . _'

app.ss_sheet.canvas.figure.update(2, True, 'dashdot')
app.ss_sheet.canvas.figure.update(3, True, 'dashdot')



# NeuralNetwork
app.nn_sheet.mv_select.v_model = ['F_cw']
app.nn_sheet.cv_select.v_model = ['T1', 'T2', 'CA1', 'CA2']

app.nn_sheet.panel.identify_model_btn.disabled = False
app.nn_sheet.panel.validate_model_btn.disabled = False

app.nn_sheet.canvas.create(train_df=nn_train_results, validation_df=nn_validation_results)

app.nn_sheet.canvas.table.table_header.checkbox_header.v_model = True

app.nn_sheet.canvas.table.items_list[4].style_item.v_model = '_ . _'
app.nn_sheet.canvas.table.items_list[5].style_item.v_model = '_ . _'
app.nn_sheet.canvas.table.items_list[6].style_item.v_model = '_ . _'
app.nn_sheet.canvas.table.items_list[7].style_item.v_model = '_ . _'

app.nn_sheet.canvas.figure.update(4, True, 'dashdot')
app.nn_sheet.canvas.figure.update(5, True, 'dashdot')
app.nn_sheet.canvas.figure.update(6, True, 'dashdot')
app.nn_sheet.canvas.figure.update(7, True, 'dashdot')



# Trasfer Function
app.tf_sheet.import_sheet.mv_select.v_model = ['F_cw', 'CA1']
app.tf_sheet.import_sheet.cv_select.v_model = ['T1', 'T2']
app.tf_sheet.import_sheet.train_condition_select.v_model = ['Training Condition']
app.tf_sheet.import_sheet.valid_condition_select.v_model = ['Validation Condition']

mv, cv = app.tf_sheet.import_sheet.get_data()

app.tf_sheet.matrix_sheet = TransferMatrix()
app.tf_sheet.matrix_sheet.create_matrix(signal_df=app.signal_df[mv + cv], capsule_df=app.capsule_df,
                                mv=mv, cv=cv,
                                train_capsules=app.tf_sheet.import_sheet.train_condition_select.v_model,
                                valid_capsules=app.tf_sheet.import_sheet.valid_condition_select.v_model)

app.tf_sheet.app_bar.next_btn.disabled = False
app.tf_sheet.matrix_sheet.matrix_dic['T1']['CA1'].turn_off_tf_card()
app.tf_sheet.matrix_sheet.matrix_dic['T2']['CA1'].turn_off_tf_card()
chip_t1 = app.tf_sheet.matrix_sheet.matrix_dic['T1']['F_cw']
chip_t2 = app.tf_sheet.matrix_sheet.matrix_dic['T2']['F_cw']
chip_t1.tf_card.deadtime_checkbox.v_model = True
chip_t2.tf_card.deadtime_checkbox.v_model = True
app.tf_sheet.window.v_model = 2
app.tf_sheet.update_nav()

# Display App
display(app.run())

# Tf Runs in the background after App Disp. 
app.tf_sheet.matrix_sheet.run()
app.tf_sheet.matrix_window.children = [app.tf_sheet.matrix_sheet]

App(children=[AppBar(children=[Tabs(background_color='grey lighten-4', children=[HamburgerMenu(children=[List(…